In [ ]:
import os
import numpy as np
from scipy import io as sio
import pandas as pd
from shutil import copyfile

In [ ]:
raw = sio.loadmat('./datasets/raw/wiki/wiki/wiki.mat')

In [ ]:
type(raw)

In [ ]:
raw.keys()

In [ ]:
raw = raw['wiki']

In [ ]:
type(raw)

In [ ]:
raw.shape

In [ ]:
raw = raw[0][0]

In [ ]:
type(raw)

In [ ]:
raw.shape

In [ ]:
len(raw)

In [ ]:
data = []
for i in raw:
    data.append(i)
data = np.array(data)

In [ ]:
data.shape

In [ ]:
data = data.squeeze()

In [ ]:
data.shape

In [ ]:
data = data.T
data.shape

In [ ]:
df = pd.DataFrame(data)
df.columns = ['dob', 'photo_taken', 'full_path', 'gender', 'name', 'face_location', 'face_score', 'second_face_score']

In [ ]:
df = df[df['face_score'] != float('-inf')]

In [ ]:
df = df.dropna(subset=['gender'])

In [ ]:
df = df[pd.isnull(df['second_face_score'])]

In [ ]:
from datetime import datetime, timedelta
def mat_date_convert(mat_date):
    return (timedelta(days=mat_date -366)+datetime(1,1,1)).year

In [ ]:
df['age'] = df['photo_taken'] - df['dob'].apply(mat_date_convert)

In [ ]:
proccesed_data = df[['full_path', 'gender', 'age']]

In [ ]:
proccesed_data

In [ ]:
# Makes un-eqal sized samples for males and females, probably better to use next block
# To run again uncomment

# female_path = './proccessed_data/gender/female/'
# male_path = './proccessed_data/gender/male/'
# wiki_path = './wiki_crop/'
# for i, row in proccesed_data.iterrows():
#     path = row['full_path'][0]
#     name = path.replace('/', '_')
#     old_path = wiki_path + path
#     age = row['age']
#     gender = row['gender']
#     if gender == 0: # female
#         new_path = female_path + name
#     else: # male
#         new_path = male_path + name
#     copyfile(old_path, new_path)

In [ ]:
# Makes eqaly sized samples for males and females
# To run again uncomment code below

train_directory = './datasets/equal_data/gender/train/'
valid_directory = './datasets/equal_data/gender/valid/'

# creates directory if not there
# Train:
if not os.path.exists(train_directory + 'male'):
    os.makedirs(train_directory + 'male')
if not os.path.exists(train_directory + 'female'):
    os.makedirs(train_directory + 'female')
# Validate:
if not os.path.exists(valid_directory + 'male'):
    os.makedirs(valid_directory + 'male')
if not os.path.exists(valid_directory + 'female'):
    os.makedirs(valid_directory + 'female')
    
wiki_path = './datasets/raw/wiki_crop/wiki_crop/'

num_male = int(sum(proccesed_data['gender']))
num_female = len(proccesed_data) - num_male
num_samples = min(num_female, num_male)

female_samples = proccesed_data[proccesed_data['gender'] == 0]
male_samples = proccesed_data[proccesed_data['gender'] == 1]

valid_percent = 10

train_female, val_female = female_samples[num_samples//valid_percent:num_samples], female_samples[:num_samples//valid_percent]
train_male, val_male = male_samples[num_samples//valid_percent:num_samples], male_samples[:num_samples//valid_percent]

def copy_data(data, gender, max_samples, origin_path, dest_path):
    copied = 0
    for _, row in data.iterrows():
        if copied == max_samples:
            break
        path = row['full_path'][0]
        name = path.replace('/', '_')
        old_path = origin_path + path
        new_path = dest_path + gender +'/' + name
        copyfile(old_path, new_path)
        copied += 1

copy_data(train_female, 'female', num_samples, wiki_path, train_directory)
copy_data(val_female, 'female', num_samples, wiki_path, valid_directory)
copy_data(train_male, 'male', num_samples, wiki_path, train_directory)
copy_data(val_male, 'male', num_samples, wiki_path, valid_directory)
    

    
